In [1]:
import pickle
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.functional import F
from torch.utils.data import Dataset, DataLoader, Subset
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence
from torch import optim
from sklearn.model_selection import GroupKFold
from sklearn.metrics import classification_report
import sklearn
from sklearn.preprocessing import MinMaxScaler
from numpy import random
from model_architecture import MatchPredictor
from gru_model_architecture import GRUMatchPredictor
import itertools
from sklearn.metrics import accuracy_score
pd.options.display.max_rows = 50

In [2]:
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
device

device(type='cuda')

In [3]:
with open('time_series_dataset.pkl', 'rb') as file:
    data = pickle.load(file)

In [4]:
class StatsDataset(Dataset):

    def __init__(self, x, y, cv=False, n_splits=None):
        self.x = x
        self.y = y
        self._cv = cv
        self._n_splits = n_splits
        if self._cv:
            self.folds = self._get_folds()

    def __getitem__(self, index):
        static_df, home_ts, away_ts = self.x[index]
        y = self.y[index]

        home = home_ts.drop(['season'], axis=1)
        away = away_ts.drop(['season'], axis=1)
        cols = ['goals_scored_ratio', 'goals_conceded_ratio', 'raiting_ratio', 'xG_ratio']
        static = static_df[static_df.columns.difference(cols)]
        
        static = torch.as_tensor(static.values, dtype=torch.float32)
        home = torch.as_tensor(home.values, dtype=torch.float32)
        away = torch.as_tensor(away.values, dtype=torch.float32)

        target = torch.as_tensor(y, dtype=torch.long)
        return static, home, away, target
    
    def __len__(self):
        return len(self.x)
    
    def _get_folds(self):
        folds = [[] for _ in range(self._n_splits)]
        for i, data in enumerate(self.x):
            folds[data[1]['season'].head(1).values[0].astype('int') % self._n_splits].append(i)
        return folds

    def get_subsets(self, val_idx):
        assert self._cv, (
        'Cross validation is not possible when _cv attribute is set to False (that means it is a testing dataset).')

        train_indices = list(itertools.chain.from_iterable(self.folds[:val_idx] + self.folds[val_idx+1:]))
        val_indices = self.folds[val_idx]
        train_data = Subset(self, train_indices)
        val_data = Subset(self, val_indices)
        return train_data, val_data

In [5]:
seasons = [key[2] for key in data.keys()]
max_season = sorted(seasons, reverse=True)[0]

train_seasons = np.arange(1, max_season, 1)
test_seasons = np.random.choice(train_seasons, size=10, replace=False)
train_seasons = np.setdiff1d(train_seasons, test_seasons)

train_data = [value for key, value in data.items() if key[2] in train_seasons]
test_data = [value for key, value in data.items() if key[2] in test_seasons]

y_train = np.array([x[0].pop('result') for x in train_data])
y_test = np.array([x[0].pop('result') for x in test_data])


In [6]:
for values in data.values():
    values[1].drop(['away_name', 'home_name', 'match_date'], inplace=True, axis=1)
    values[2].drop(['away_name', 'home_name', 'match_date'], inplace=True, axis=1)

    values[1].fillna({'round' : 55, 'coach_matches' : 0}, inplace=True)
    values[2].fillna({'round' : 55, 'coach_matches' : 0}, inplace=True)

    '''values[1] = values[1][['result_from_team_perspective', 'season']]
    values[2] = values[2][['result_from_team_perspective', 'season']]'''

In [7]:
statics_lst, ts_homes_lst, ts_aways_lst = zip(*train_data)

home_dfs = pd.concat(ts_homes_lst)
home_dfs.drop('season', axis=1, inplace=True)
away_dfs = pd.concat(ts_aways_lst)
away_dfs.drop('season', axis=1, inplace=True)
static_dfs = pd.concat(statics_lst)

home_scaler = MinMaxScaler()
away_scaler  = MinMaxScaler()
static_scaler = MinMaxScaler()

home_scaler.fit(home_dfs.values)
away_scaler.fit(away_dfs.values)
static_scaler.fit(static_dfs.values)

for i, el in enumerate(train_data):
    home_seasons = el[1].pop('season')
    away_seasons = el[2].pop('season')

    train_data[i][0] = pd.DataFrame(static_scaler.transform(el[0].values), index=el[0].index, columns=el[0].columns)

    train_data[i][1] = pd.DataFrame(
        home_scaler.transform(el[1].values), index=el[1].index, columns=el[1].columns)
    
    train_data[i][2] = pd.DataFrame(
        away_scaler.transform(el[2].values), index=el[2].index, columns=el[2].columns)
    
    train_data[i][1]['season'] = home_seasons
    train_data[i][2]['season'] = away_seasons

for i, el in enumerate(test_data):
    home_seasons = el[1].pop('season')
    away_seasons = el[2].pop('season')

    test_data[i][0] = pd.DataFrame(static_scaler.transform(el[0].values), index=el[0].index, columns=el[0].columns)
    test_data[i][1] = pd.DataFrame(
        home_scaler.transform(el[1].values), index=el[1].index, columns=el[1].columns)
    
    test_data[i][2] = pd.DataFrame(
        away_scaler.transform(el[2].values), index=el[2].index, columns=el[2].columns)
    
    test_data[i][1]['season'] = home_seasons
    test_data[i][2]['season'] = away_seasons

In [8]:
#model = MatchPredictor(40, 8, 1, 4, device)
model = GRUMatchPredictor(40, 8, 1, 4, device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
model.to(device)

c:\Users\dawid\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


GRUMatchPredictor(
  (home_gru): GRU(40, 8, batch_first=True, dropout=0.2)
  (away_gru): GRU(40, 8, batch_first=True, dropout=0.2)
  (fc): Sequential(
    (0): Linear(in_features=20, out_features=32, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=32, out_features=16, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=16, out_features=3, bias=True)
  )
)

In [9]:
def collate_fn(batch):
    statics, homes, aways, targets = zip(*batch)

    away_lens = torch.as_tensor([a.size(0) for a in aways])
    home_lens = torch.as_tensor([h.size(0) for h in homes])

    home_padded = pad_sequence(homes, batch_first=True, padding_value=0.0)
    away_padded = pad_sequence(aways, batch_first=True, padding_value=0.0)

    home_packed = pack_padded_sequence(home_padded, home_lens, batch_first=True, enforce_sorted=False)
    away_packed = pack_padded_sequence(away_padded, away_lens, batch_first=True, enforce_sorted=False)

    statics = torch.stack(statics)
    statics = statics.squeeze(1)
    targets = torch.stack(targets)

    return {
        'statics' : statics,
        'homes' : home_packed,
        'aways' : away_packed,
        'targets' : targets
    }


In [10]:
n_splits = 5
batch_size = 64
num_epochs = 4

train_dataset = StatsDataset(train_data, y_train, cv=True, n_splits=n_splits)
test_dataset = StatsDataset(test_data, y_test)
test_loader = DataLoader(
    test_dataset, batch_size=batch_size, num_workers=0, collate_fn=collate_fn)

train_losses = []
val_losses = []

for epoch in range(num_epochs):
    for val_idx in range(n_splits):
        train_fold, val_fold = train_dataset.get_subsets(val_idx)

        train_loader = DataLoader(
            train_fold, batch_size=batch_size, shuffle=False, num_workers=0, collate_fn=collate_fn)
        
        val_loader = DataLoader(
            val_fold, batch_size=batch_size, num_workers=0, collate_fn=collate_fn)

        model.train()
        running_loss = 0.0
        for batch in train_loader:
            optimizer.zero_grad()
            statics, homes, aways = batch['statics'].to(device), batch['homes'].to(device), batch['aways'].to(device)
            targets = batch['targets'].squeeze(1).to(device)

            logits = model(statics, homes, aways)
            loss = criterion(logits, targets)
            running_loss += loss.item() * homes.data.size(0)

            loss.backward()
            optimizer.step()

        total_train_loss = running_loss / len(train_loader.dataset)
        train_losses.append(total_train_loss)

        model.eval()
        running_loss = 0.0
        with torch.no_grad():
            for batch in val_loader:
                statics, homes, aways = batch['statics'].to(device), batch['homes'].to(device), batch['aways'].to(device)
                targets = batch['targets'].squeeze(1).to(device)

                logits = model(statics, homes, aways)
                loss = criterion(logits, targets)
                running_loss += loss.item() * homes.data.size(0)

            total_val_loss = running_loss / len(val_loader.dataset)
            val_losses.append(total_val_loss)

            pred = F.softmax(logits, dim=1).argmax(dim=1)

    print(f'Epoch: {epoch+1}/{num_epochs} - train loss: {train_losses[-1]} - validation loss: {val_losses[-1]}')


Epoch: 1/4 - train loss: 20.874109897128534 - validation loss: 20.201545101986415
Epoch: 2/4 - train loss: 20.62457866252745 - validation loss: 19.986841885057537
Epoch: 3/4 - train loss: 20.582863001378712 - validation loss: 19.91410976309049
Epoch: 4/4 - train loss: 20.509870352561236 - validation loss: 19.80500563422057


In [11]:
test_dataset = StatsDataset(test_data, y_test)
test_loader = DataLoader(
    test_dataset, batch_size=20000, num_workers=0, collate_fn=collate_fn)

model.eval()
with torch.no_grad():
    for batch in test_loader:
        statics, homes, aways = batch['statics'].to(device), batch['homes'].to(device), batch['aways'].to(device)
        targets = batch['targets'].squeeze(1).to(device)

        logits = model(statics, homes, aways)
        pred = F.softmax(logits, dim=1).argmax(dim=1)
        print(classification_report(targets.cpu(), pred.cpu()))
        
        score = accuracy_score(targets.cpu(), pred.cpu())
        

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       701
           1       0.50      0.83      0.63      1170
           2       0.48      0.45      0.47       825

    accuracy                           0.50      2696
   macro avg       0.33      0.43      0.36      2696
weighted avg       0.37      0.50      0.42      2696



c:\Users\dawid\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\dawid\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\dawid\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [12]:
rrr = StatsDataset(train_data, y_train)
rrr_loader = DataLoader(
    rrr, batch_size=20000, num_workers=0, collate_fn=collate_fn)

model.eval()
with torch.no_grad():
    for batch in rrr_loader:
        statics, homes, aways = batch['statics'].to(device), batch['homes'].to(device), batch['aways'].to(device)
        targets = batch['targets'].squeeze(1).to(device)

        logits = model(statics, homes, aways)
        pred = F.softmax(logits, dim=1).argmax(dim=1)
        print(classification_report(y_train, pred.cpu()))
        
        

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00      4440
         1.0       0.50      0.82      0.62      7532
         2.0       0.46      0.42      0.44      5258

    accuracy                           0.48     17230
   macro avg       0.32      0.41      0.35     17230
weighted avg       0.36      0.48      0.40     17230



c:\Users\dawid\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\dawid\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\dawid\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [ ]:
# save GRU model
#torch.save(model.state_dict(), f'gru_model_{score}.pkl')